<a href="https://colab.research.google.com/github/BaseKan/aiday_training_resources/blob/main/Cython/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext Cython
import numpy as np


Make It Work Make It Right Make It Fast

In [39]:
%%cython

def c_fib_typed(int n):
  """Return the Fibonacci series up to n."""
  cdef int a, b, len_r
  cdef int r[1000]
  a = 0
  b = 1
  len_r = 0
  while b < n:
    r[len_r] = b
    len_r += 1
    a, b = b, a + b
        
  r_as_list  = [item for item in r[:len_r]]
  return r_as_list

In [40]:
%timeit x = c_fib_typed(2000)

The slowest run took 10.85 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 5: 285 ns per loop


# Cython and Numpy Arrays

UITLEG MEMORY VIEWS


In [9]:
%%cython
import numpy as np

arr = np.arange(9, dtype=np.dtype("i")).reshape((3, 3))
print(arr)

cdef int [:, :] arr_view = arr
print(np.asarray(arr_view))

[[0 1 2]
 [3 4 5]
 [6 7 8]]
[[0 1 2]
 [3 4 5]
 [6 7 8]]


In [10]:
%%cython 
import numpy as np

cdef int [:, :] arr_view = np.arange(9, dtype=np.dtype("i")).reshape((3, 3))
print(np.asarray(arr_view))
print(np.asarray(arr_view[1,1]))
arr_view[1,1] = 10
print(np.asarray(arr_view[1,1]))
arr_view[:,:] = 5
print(np.asarray(arr_view))


[[0 1 2]
 [3 4 5]
 [6 7 8]]
4
10
[[5 5 5]
 [5 5 5]
 [5 5 5]]


## Looping over arrays

UITLEG FOR LOOPS IN C

In [31]:
x = np.arange(100, dtype=np.dtype("i")).reshape((10, 10))
y = np.full((10,10), 2, dtype=np.dtype("i"))

In [36]:
def array_op(x,y):
  result = np.zeros(x.shape)
  for i in range(x.shape[0]):
      for j in range(x.shape[1]):
          result[i,j] = x[i,j] * y[i,j] + i
  return result
    

In [37]:
%timeit result = array_op(x,y)

1000 loops, best of 5: 257 µs per loop


In [7]:
%%cython 
import numpy as np

def cdef_array_op(x, y):
    result = np.zeros(x.shape)
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            result[i,j] = x[i,j] * y[i,j] + i
    return result

In [8]:
%timeit cdef_array_op(x,y)

1000 loops, best of 5: 274 µs per loop


In [12]:
%%cython 
import numpy as np

def typed_array_op(int[:,:] x, int[:,:] y):
    cdef int[:,:] result = np.zeros((x.shape[0], x.shape[1]), dtype = np.dtype("i"))
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            result[i,j] = x[i,j] * y[i,j] + i
    return result

In [14]:
%timeit typed_array_op(x,y)

The slowest run took 8.50 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 5.82 µs per loop


In [29]:
%%cython 
import numpy as np
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
def typed_unsafe_array_op(int[:,:] x, int[:,:] y):
    cdef int[:,:] result = np.zeros((x.shape[0], x.shape[1]), dtype = np.dtype("i"))
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            result[i,j] = x[i,j] * y[i,j] + i
    return result


In [32]:
%timeit typed_unsafe_array_op(x,y)

The slowest run took 11.65 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 4.5 µs per loop


# Cython and Pandas

We downloaden eerst wat data...

In [2]:
!curl -L -c cookies.txt 'https://docs.google.com/uc?export=download&id=151gCztjHR_D2uIoebxfi52DZWGLabOQd' | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1/p' > confirm.txt    
!curl -L -b cookies.txt -o 'weatherAUS.zip' 'https://docs.google.com/uc?export=download&id=151gCztjHR_D2uIoebxfi52DZWGLabOQd&confirm='$(<confirm.txt)
!unzip weatherAUS.zip
!rm -f confirm.txt cookies.txt weatherAUS.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    195      0 --:--:--  0:00:01 --:--:--   195
100 3781k    0 3781k    0     0  1524k      0 --:--:--  0:00:02 --:--:-- 1524k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0    218      0 --:--:--  0:00:01 --:--:--   218
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
100 3781k    0 3781k    0     0  1576k      0 --:--:--  0:00:02 --:--:-- 1576k
Archive:  weatherAUS.zip
  inflating: weatherAUS.csv          


In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('weatherAUS.csv')

In [15]:
df = df.apply(lambda x: x.fillna(x.mean()) if x.dtype == 'float64' else x,
              axis=0)
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,5.468232,7.611178,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.000000,4.50993,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,5.468232,7.611178,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,4.447461,4.50993,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,5.468232,7.611178,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,4.447461,2.00000,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,5.468232,7.611178,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,4.447461,4.50993,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,5.468232,7.611178,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.000000,8.00000,17.8,29.7,No,No


In [16]:
def average_value(values):
  return np.mean(values)


In [17]:
%timeit df.head(100).apply(lambda x: average_value(x[11:13]), axis=1)


10 loops, best of 5: 20.6 ms per loop


In [19]:
%%cython
import numpy as np

def c_average_value(values):
  return np.mean(values)


In [21]:
%timeit df.head(100).apply(lambda x: c_average_value(x[11:13]), axis=1)

100 loops, best of 5: 17 ms per loop


In [22]:
%%cython
import numpy as np
cimport numpy as np

def c_average_value_typed(int[:] values):
  return np.mean(values)

In [24]:
%timeit df.head(100).apply(lambda x: c_average_value_typed(x[11:13].to_numpy(dtype=np.dtype('i'))), axis=1)

100 loops, best of 5: 6.23 ms per loop


TEKST OVER PROFILING

In [26]:
%%cython
cimport cython
cimport numpy as np
import numpy as np

cdef int c_average_value_typed(int[:] values):
  return np.mean(values)

def c_apply_average_value(int[:,:] df_cols):
  n = df_cols.shape[0]
  cdef double[:] res = np.empty(n)
  for i in range(n):
    res[i] = c_average_value_typed(df_cols[i])

  return res

In [27]:
%timeit c_apply_average_value(df.head(100)[df.columns[11:13]].to_numpy(dtype=np.dtype('i'))) 

1000 loops, best of 5: 1.48 ms per loop
